In [9]:
import pandas as pd
import numpy as np
import copy
from time import time

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [3]:
from math import sin, cos, sqrt, atan2, radians

def get_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return round(distance, 2)

In [4]:
def get_time(time_str):
#     v = int(data1["starttime"][i][11:13])
    int_time = int(time_str[11:13])
    if 0 <= int_time and int_time < 6:
        return "night"
    if 6 <= int_time and int_time < 12:
        return "morning"
    if 12 <= int_time and int_time < 18:
        return "afternoon"
    if 18 <= int_time and int_time <= 23:
        return "evening"

In [7]:
data = pd.read_csv('data/2018/201806-citibike-tripdata.csv')

In [10]:
from sklearn.utils import shuffle

cc=time()
data = shuffle(data)
print(time() - cc)

56.77583074569702


In [11]:
print(len(data))

1953103


In [63]:
data1 = copy.deepcopy(data)
# data1 = data[0:1000]

In [64]:
print(len(data1))

1953103


In [65]:
data1.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
1215566,615,2018-06-26 10:51:37.2460,2018-06-26 11:01:52.5190,3069,Lorimer St & Broadway,40.704118,-73.948186,3092,Berry St & N 8 St,40.719009,-73.958525,28426,Subscriber,1989,2
893233,1958,2018-06-25 13:56:24.1950,2018-06-25 14:29:02.2030,490,8 Ave & W 33 St,40.751551,-73.993934,3002,South End Ave & Liberty St,40.711512,-74.015756,30010,Subscriber,1959,1
1639866,1385,2018-06-14 11:11:57.6520,2018-06-14 11:35:03.5190,3374,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613,2006,Central Park S & 6 Ave,40.765909,-73.976342,28491,Customer,1969,0
573389,910,2018-06-16 17:41:18.5640,2018-06-16 17:56:28.7240,405,Washington St & Gansevoort St,40.739323,-74.008119,515,W 43 St & 10 Ave,40.760094,-73.994618,32969,Subscriber,1967,1
1076535,1413,2018-06-19 15:59:59.9820,2018-06-19 16:23:33.4060,528,2 Ave & E 31 St,40.742909,-73.977061,530,11 Ave & W 59 St,40.771522,-73.990541,25498,Subscriber,1983,1


In [66]:
from time import time
new_times = [" "] * len(data) 
distances = [0] * len(data)
print(len(new_times))
c=time()

for i, row in data1.iterrows():
#     print(i)
    new_times[i] = get_time(row["starttime"])
    distances[i] = get_distance(row["start station latitude"], row["start station longitude"], row["end station latitude"], row["end station longitude"])
    
data1["time"] = pd.Series(new_times)
data1["distance"] = pd.Series(distances)

print(time() - c)

1953103
380.2947733402252


In [67]:
del data1["starttime"]
del data1["stoptime"]
del data1["start station id"]
del data1["start station name"]
del data1["end station id"]
del data1["end station name"]
del data1["bikeid"]
del data1["start station latitude"]
del data1["start station longitude"]
del data1["end station latitude"]
del data1["end station longitude"]

In [68]:
data1.head(5)

,tripduration,usertype,birth year,gender,time,distance
1215566,615,Subscriber,1989,2,morning,1.87
893233,1958,Subscriber,1959,1,afternoon,4.82
1639866,1385,Customer,1969,0,morning,4.12
573389,910,Subscriber,1967,1,afternoon,2.58
1076535,1413,Subscriber,1983,1,afternoon,3.38


In [52]:
cc=time()
# data1 = shuffle(data1)
print(time() - cc)

0.0


In [69]:
data1 = data1.reset_index(drop=True)
data1.head(5)

,tripduration,usertype,birth year,gender,time,distance
0,615,Subscriber,1989,2,morning,1.87
1,1958,Subscriber,1959,1,afternoon,4.82
2,1385,Customer,1969,0,morning,4.12
3,910,Subscriber,1967,1,afternoon,2.58
4,1413,Subscriber,1983,1,afternoon,3.38


In [79]:
data1.to_csv("./data/2018/processed.csv", index=False)

In [70]:
k = int(0.8*len(data1))
print(k)
x_train = data1["tripduration"][:k]
x_test = data1["tripduration"][k:]
y_train = data1["distance"][:k]
y_test = data1["distance"][k:]

1562482


In [71]:
print(len(x_train), len(x_test))

1562482 390621


In [72]:
model = LinearRegression()

model.fit(list(map(lambda x:[x],x_train)), y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [73]:
predict = model.predict(list(map(lambda x:[x],x_test)))

In [74]:
print(model.coef_)

[2.09971723e-06]


In [75]:
print(metrics.mean_squared_error(y_test, predict))

2.257754180699111


In [76]:
print(metrics.r2_score(y_test, predict))

0.0010547559555954544


In [77]:
model.score(list(map(lambda x:[x],x_train)), y_train)

0.0012937168614759467

In [78]:
model.score(list(map(lambda x:[x],x_test)), y_test)

0.0010547559555954544